# 第七章：构建聊天应用
## OpenAI API 快速入门

本笔记本改编自 [Azure OpenAI 示例仓库](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst)，该仓库包含了访问 [Azure OpenAI](notebook-azure-openai.ipynb) 服务的相关笔记本。

Python 的 OpenAI API 也可以用于 Azure OpenAI 模型，只需做一些小的修改。你可以在这里了解两者的区别：[如何在 Python 中切换 OpenAI 和 Azure OpenAI 端点](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# 概述  
“大型语言模型是一种将文本映射为文本的函数。给定一段输入文本，大型语言模型会尝试预测接下来会出现的文本”(1)。本“快速入门”笔记本将为用户介绍高级的LLM概念、开始使用AML所需的核心包、提示词设计的基础知识，以及几个不同应用场景的简短示例。


## 目录

[概述](../../../../07-building-chat-applications/python)  
[如何使用 OpenAI 服务](../../../../07-building-chat-applications/python)  
[1. 创建你的 OpenAI 服务](../../../../07-building-chat-applications/python)  
[2. 安装](../../../../07-building-chat-applications/python)  
[3. 凭证](../../../../07-building-chat-applications/python)  

[使用场景](../../../../07-building-chat-applications/python)  
[1. 文本摘要](../../../../07-building-chat-applications/python)  
[2. 文本分类](../../../../07-building-chat-applications/python)  
[3. 生成新产品名称](../../../../07-building-chat-applications/python)  
[4. 微调分类器](../../../../07-building-chat-applications/python)  

[参考资料](../../../../07-building-chat-applications/python)


### 构建你的第一个提示词  
这个简短的练习将为你介绍如何向 OpenAI 模型提交提示词来完成一个简单的任务——“摘要”。

**步骤**：  
1. 在你的 python 环境中安装 OpenAI 库  
2. 加载常用的辅助库，并为你创建的 OpenAI 服务设置常用的安全凭证  
3. 为你的任务选择一个模型  
4. 为模型创建一个简单的提示词  
5. 向模型 API 提交你的请求！


### 1. 安装 OpenAI


In [ ]:
%pip install openai python-dotenv

### 2. 导入辅助库并实例化凭证


In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. 寻找合适的模型  
GPT-3.5-turbo 或 GPT-4 模型能够理解并生成自然语言。


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. 提示词设计

“大型语言模型的神奇之处在于，通过在海量文本上训练以最小化预测误差，模型最终学会了对这些预测有用的概念。例如，它们学会了以下内容”(1)：

* 如何拼写
* 语法是如何运作的
* 如何改写句子
* 如何回答问题
* 如何进行对话
* 如何用多种语言写作
* 如何编写代码
* 等等

#### 如何控制大型语言模型  
“在所有输入中，对大型语言模型影响最大的是文本提示词(1)。

可以通过几种方式提示大型语言模型生成输出：

指令：告诉模型你想要什么  
补全：引导模型补全你想要的开头  
示例：通过以下方式向模型展示你想要的内容：  
在提示词中给出几个示例  
在微调训练数据集中提供数百或数千个示例”



#### 创建提示词有三条基本准则：

**展示并说明**。通过指令、示例或两者结合，让模型清楚你想要什么。如果你希望模型按字母顺序排列一组项目，或按情感对一段文字进行分类，就要明确地展示你的需求。

**提供高质量数据**。如果你想构建一个分类器或让模型遵循某种模式，确保有足够的示例。一定要仔细检查你的示例——模型通常足够聪明，可以识别基本的拼写错误并给出回应，但它也可能认为这些错误是有意为之，这会影响输出结果。

**检查你的设置。** temperature 和 top_p 设置控制模型生成回复时的确定性。如果你需要模型给出唯一正确的答案，应该把这些参数调低。如果你希望得到更多样化的回复，可以把它们调高。人们在使用这些设置时最常犯的错误，就是以为它们是“聪明度”或“创造力”的调节器。

来源：https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### 重复相同的调用，结果有何不同？


In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## 总结文本  
#### 挑战  
通过在文本段落末尾添加“tl;dr:”来对文本进行总结。请注意，模型能够在没有额外指令的情况下完成多种任务。你可以尝试使用比“tl;dr”更具描述性的提示词，以调整模型的行为并自定义你获得的摘要(3)。

最近的研究表明，通过在大规模文本语料库上进行预训练，然后针对特定任务进行微调，可以在许多NLP任务和基准测试中取得显著提升。虽然这种方法在架构上通常与任务无关，但仍然需要成千上万个任务相关的微调数据集。相比之下，人类通常只需几个例子或简单的指令就能完成新的语言任务——而当前的NLP系统在这方面仍然存在很大困难。我们在这里展示，扩大语言模型的规模可以极大提升与任务无关的少样本表现，有时甚至能与以往最先进的微调方法相媲美。



Tl;dr


# 多种用例的练习  
1. 总结文本  
2. 文本分类  
3. 生成新产品名称


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 文本分类  
#### 挑战  
将项目分类到推理时提供的类别中。在下面的例子中，我们在提示中同时提供了类别和需要分类的文本（*playground_reference）。

客户咨询：你好，我的笔记本键盘有一个按键最近坏了，我需要更换：

分类结果：


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 生成新产品名称
#### 挑战
根据示例词语创造产品名称。在提示中，我们会提供需要命名的产品信息。同时也会给出类似的示例，以展示我们期望的命名模式。我们还将温度值设置得较高，以增加随机性和创新性。

产品描述：一款家用奶昔机  
种子词：快速、健康、紧凑  
产品名称：HomeShaker、Fit Shaker、QuickShake、Shake Maker

产品描述：一双可以适合任何脚型的鞋子  
种子词：适应、合脚、全适配


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# 参考资料  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio 示例](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [微调 GPT-3 进行文本分类的最佳实践](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# 获取更多帮助  
[OpenAI 商业化团队](AzureOpenAITeam@microsoft.com)


# 贡献者
* Louis Li



---

**免责声明**：  
本文件由 AI 翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻译。我们力求准确，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文件应被视为权威来源。对于关键信息，建议使用专业人工翻译。因使用本翻译而产生的任何误解或曲解，我们概不负责。
